![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [1]:
!pip install transformers
!pip install evaluate

from transformers import logging
logging.set_verbosity(logging.WARNING)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Load the dataset
import pandas as pd
df = pd.read_csv("data/car_reviews.csv", delimiter=";")

In [3]:
df.head()

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


### Question 1: Sentiment classification

In [4]:
# Build a classifier model
from transformers import pipeline

classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
# Converting the input text to a list
text = df["Review"].tolist()
class_labels = df["Class"].tolist()

In [7]:
# Generate the predictions
predicted_labels = classifier(text)
for text_review, prediction, label in zip(text, predicted_labels, class_labels):
    print(f"Review: {text_review} \nReal Sentiment: {label}\nPredicted Sentiment: {prediction['label']} (Confidence: {prediction['score']:.3f})\n")

Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

In [8]:
# Converting the true labels to 1s and 0s
true_labels = [1 if class_label == "POSITIVE" else 0 for class_label in class_labels]

In [9]:
# Generate. the predicted labels
predictions = [1 if pred["label"] == "POSITIVE" else 0 for pred in predicted_labels]

In [14]:
# Calculating the accuracy score
import evaluate

accuracy = evaluate.load("accuracy")
accuracy_result_dict = accuracy.compute(references=true_labels, predictions=predictions)
accuracy_result = accuracy_result_dict["accuracy"]
print(f"Accuracy: {accuracy_result}")

Accuracy: 0.8


In [13]:
# Calculating the f1 score

f1 = evaluate.load("f1")
f1_result_dict = f1.compute(references=true_labels, predictions=predictions)
f1_result = f1_result_dict['f1']
print(f"F1 result: {f1_result}")

F1 result: 0.8571428571428571


### Question 2: Translation

In [16]:
# Build a translation model 
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

In [17]:
# Load the input text and translate the car reviews
input_text = text[0]
translated_outputs = translator(input_text, max_length=27)
translated_review = translated_outputs[0]["translation_text"]
print(f"Model Translation: {translated_review}")

Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Model Translation: Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


In [19]:
# Load reference translations from file
with open("data/reference_translations.txt", 'r') as file:
    lines = file.readlines()
references = [line.strip() for line in lines]
print(f"Spanish translation references: \n{references}")

Spanish translation references: 
['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']


In [21]:
# Evaluate the BLEU score
bleu = evaluate.load("bleu")

bleu_score = bleu.compute(predictions=[translated_review], references=[references])
bleu_score

{'bleu': 0.6022774485691839,
 'precisions': [0.9090909090909091,
  0.7142857142857143,
  0.55,
  0.3684210526315789],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0476190476190477,
 'translation_length': 22,
 'reference_length': 21}

### Question 3: Extractive QA

In [23]:
# Load an extractive QA LLM
llm_QA = pipeline("question-answering", model="deepset/minilm-uncased-squad2")

In [24]:
# Generate the context and question
context = text[1]
question = "What did he like about the brand?"

# Get the outputs
outputs = llm_QA(context=context, question=question)

answer = outputs["answer"]
print(f"Answer: {answer}")

Answer: ride quality, reliability


### Question 4: Summarization

In [26]:
# Summarize the last review
from transformers import pipeline

# Load the pre-trained model
llm_sum = pipeline("summarization", model="cnicu/t5-small-booksum")
last_review = text[-1]
print(f"Original text: \n{last_review}")

# Output the summary
outputs_sum = llm_sum(last_review, max_length=52, clean_up_tokenization_spaces=True)
summarized_text = outputs_sum[0]["summary_text"]
print(f"Summarized text: \n{summarized_text}")

Original text: 
I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of l